In [1]:
import pickle
import requests
from bs4 import BeautifulSoup

In [2]:
rev = "https://revistas.urp.edu.pe/"
req = requests.get(rev)
soup = BeautifulSoup(req.content, "lxml")

In [3]:
journal = soup.find_all("li", attrs="has_thumb")[14]
link_journal = journal.find("a")["href"]
req_journal = requests.get(link_journal)
soup_journal = BeautifulSoup(req_journal.content, "lxml")
link_vol = soup_journal.find("a", attrs="read_more")["href"]

In [4]:
req_vol = requests.get(link_vol)
soup_vol = BeautifulSoup(req_vol.content, "lxml")
soup_vols = soup_vol.find("ul", "issues_archive").find_all("div", attrs="obj_issue_summary")
soup_vols

[<div class="obj_issue_summary">
 <a class="cover" href="https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/291">
 <img src="https://revistas.urp.edu.pe/public/journals/6/cover_issue_291_es_ES.jpg"/>
 </a>
 <a class="title" href="https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/291">
 					Perfiles de Ingeniería
 			</a>
 <div class="series">
 			Vol. 17 Núm. 17 (2021)
 		</div>
 <div class="description">
 </div>
 </div>,
 <div class="obj_issue_summary">
 <a class="cover" href="https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/51">
 <img alt="tics" src="https://revistas.urp.edu.pe/public/journals/6/cover_issue_51_es_ES.jpg"/>
 </a>
 <a class="title" href="https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/51">
 					Perfiles de Ingeniería
 			</a>
 <div class="series">
 			Vol. 16 Núm. 16 (2020)
 		</div>
 <div class="description">
 <p> </p>
 <p> </p>
 </div>
 </div>,
 <div class="obj_issue_summary">
 <a class="

In [5]:
vols = []
for volu in soup_vols:
    dictt = {}
    series = volu.find("div", attrs="series").string.strip()
    dictt["archive"] = series.replace(".", "").replace(" ", "_").replace("(","").replace(")","").replace("ú","u")
    dictt["link_vol"] = volu.find("a")["href"]
    dictt["title"] = series
    vols.append(dictt)
vols

[{'archive': 'Vol_17_Num_17_2021',
  'link_vol': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/291',
  'title': 'Vol. 17 Núm. 17 (2021)'},
 {'archive': 'Vol_16_Num_16_2020',
  'link_vol': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/51',
  'title': 'Vol. 16 Núm. 16 (2020)'},
 {'archive': 'Vol_15_Num_15_2019',
  'link_vol': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/256',
  'title': 'Vol. 15 Núm. 15 (2019)'},
 {'archive': 'Vol_14_Num_14_2018',
  'link_vol': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/198',
  'title': 'Vol. 14 Núm. 14 (2018)'},
 {'archive': 'Vol_13_Num_13_2017',
  'link_vol': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/139',
  'title': 'Vol. 13 Núm. 13 (2017)'},
 {'archive': 'Vol_12_Num_12_2016',
  'link_vol': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/86',
  'title': 'Vol. 12 Núm. 12 (2016)'},
 {'archive': 'Vol_11_Num

In [6]:
for volu in vols:
    req_volu = requests.get(volu["link_vol"])
    soup_volu = BeautifulSoup(req_volu.content, "lxml")
    section = soup_volu.find("div", attrs="sections").find_all("div", attrs="section")[-1]
    artt = []
    arts = section.find_all("div", attrs="obj_article_summary")
    for art in arts:
        dictt = {}
        link_art = art.find("div", attrs="title").a
        title = link_art.string
        if (title is None):
            title = link_art.contents[0]
        title = title.replace("\t", "").replace("\n","")
        link_art = link_art["href"]
        authors = art.find("div", attrs="authors").string
        authors = authors.replace("\t", "").replace("\n", "").strip().split(",")
        dictt["title"] = title
        dictt["link_art"] = link_art
        dictt["authors"] = authors
        artt.append(dictt)
    volu["articles"] = artt
# soup_arts = soup_volu.find_all("ul", attrs="cmp_article_list articles")[-1].find_all("li")[0]
# authors = soup_arts.find("div", attrs="authors").string.replace("\n","").replace("\t","").split(",")
# archive = volu["archive"] + ".pickle"
# link_art = soup_arts.a["href"]
# req_art = requests.get(link_art)
# soup_art = BeautifulSoup(req_art.content, "lxml")
# archive = volu["archive"] + "_" + "1" + ".pickle"
# with open(archive, "wb") as f:
#     pickle.dump(soup_art, f)

In [7]:
for vol in vols:
    for art in vol["articles"]:
        req_art = requests.get(art["link_art"])
        soup_art = BeautifulSoup(req_art.content, "lxml")
        date = soup_art.find("div", attrs="item published").find("div", attrs="value").string
        date = date.replace("\n", "").replace("\t", "")
        art["date"] = date

In [8]:
print(vols)

[{'archive': 'Vol_17_Num_17_2021', 'link_vol': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/291', 'title': 'Vol. 17 Núm. 17 (2021)', 'articles': [{'title': 'Las pantallas acústicas como solución a la contaminación sonora en el paradero Benavides', 'link_art': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/article/view/4573', 'authors': ['Yeampier Iñigo Pozo', ' Lucero Romero Santos', ' Esther Vargas Chang'], 'date': '2021-12-31'}, {'title': 'Resiliencia en Edificaciones con Aisladores Sísmicos', 'link_art': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/article/view/4574', 'authors': ['Frances Benites Jessen', ' Esther Vargas Chang'], 'date': '2021-12-31'}, {'title': 'Planificación y desarrollo de ciudades inteligentes utilizando herramientas “BIM”: un enfoque integral de gestión', 'link_art': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/article/view/4575', 'authors': ['Carlos Chang', ' Mario Vélez Canchanya'], 'date': '202

In [9]:
with open("scrappURP.pickle", "wb") as f:
    pickle.dump(vols, f)

In [10]:
with open("scrappURP.pickle", "rb") as f:
    obj = pickle.load(f)
    print(obj)

[{'archive': 'Vol_17_Num_17_2021', 'link_vol': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/issue/view/291', 'title': 'Vol. 17 Núm. 17 (2021)', 'articles': [{'title': 'Las pantallas acústicas como solución a la contaminación sonora en el paradero Benavides', 'link_art': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/article/view/4573', 'authors': ['Yeampier Iñigo Pozo', ' Lucero Romero Santos', ' Esther Vargas Chang'], 'date': '2021-12-31'}, {'title': 'Resiliencia en Edificaciones con Aisladores Sísmicos', 'link_art': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/article/view/4574', 'authors': ['Frances Benites Jessen', ' Esther Vargas Chang'], 'date': '2021-12-31'}, {'title': 'Planificación y desarrollo de ciudades inteligentes utilizando herramientas “BIM”: un enfoque integral de gestión', 'link_art': 'https://revistas.urp.edu.pe/index.php/Perfiles_Ingenieria/article/view/4575', 'authors': ['Carlos Chang', ' Mario Vélez Canchanya'], 'date': '202